<a href="https://colab.research.google.com/github/OskarKrafft/Machine-Learning-Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unused Chunks

In [ ]:
# Data Exploration

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(eppes.dtypes)

## Set up WD

In [ ]:
# Mount GDrive to facilitate data import
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change working directory to project folder
%cd /content/drive/MyDrive/Colab Notebooks/Machine-Learning-Project

In [ ]:
# If Github repo is not yet cloned in Drive, use following code (just do once!):

# ! git clone https://github.com/OskarKrafft/Machine-Learning-Project.git

## Import libraries

In [ ]:
# Install packages and import them

import pandas as pd
import pyreadstat

## Import data

In [ ]:
eppes, meta = pyreadstat.read_sav('data/raw/1.0-EPPES.sav')
pd.set_option('display.max_columns', None)

# Data Cleaning

## Drop non-voters

In [ ]:
# Drop observation of people that were not eligible to vote

eppes = eppes[eppes.qg1.isna() == False]

eppes

In [ ]:
# Check for NAs in vote variable qg1

eppes['qg1'].isna().sum()

In [ ]:
# Create frequency table of qg1

eppes["qg1"].value_counts()

## Drop problematic, non-complete or uninteresting columns

In [ ]:
# Import Excel sheet containing column indeces to be dropped

columns_analysis = pd.read_excel(r'Drop_Columns.xlsx')
columns_analysis = columns_analysis.drop(columns_analysis.columns[[0]], axis = 1)

columns_analysis

In [ ]:
# Create list of indeces of columns to be dropped

indeces_to_drop = []

for i in range(872):
  if columns_analysis.iloc[i, 2] == False:
    indeces_to_drop.append(i)

indeces_to_drop

In [ ]:
# Drop columns by indeces_to_drop

eppes = eppes.drop(eppes.columns[indeces_to_drop], axis = 1)

eppes

## Deal with NAs

In [ ]:
# Check for NAs in columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(eppes.isna().sum())

Result: d1, d2 & d40b contain NAs

### Next Steps

**d1**: 
*In political matters people talk of "the left" and "the right". How would you place your views on this scale?*

-> Change values of rows with d1 == NA to d1 == 12 (DK)

**d25**:
*Would you say you live in a...? Rural area or village, small or middle sized town, etc.*

-> Change values of rows with d25 == NA to d25 == 4 (DK)

**d40b**:
Could you tell me how many children less than 10 years old live in your household?

-> replace values with 0

Oskar: Is replacing them with cero really correct??

In [ ]:
# Replace values of rows with d1 == NA with d1 == 12

eppes.loc[eppes['d1'].isna(), 'd1'] = 12

# Replace values of rows with d25 == NA with d25 == 4

eppes.loc[eppes['d25'].isna(), 'd25'] = 4

# Replace values of rows with d40b == NA with d40b == 0

eppes.loc[eppes['d40b'].isna(), 'd40b'] = 0

In [ ]:
# Check for NAs in eppes
eppes.isna().sum().sum()

## Delete rows with NAs in ordinal variables

In [ ]:
eppes

In [ ]:
eppes.describe()

d1, d7 and d11 have values of 97/98?

In [ ]:
eppes["d1"].value_counts()

In [ ]:
eppes["d7"].value_counts()

In [ ]:
eppes["d11"].value_counts()

Explanation: 

- d1 and d7: DKs
- d11: true age, no changes necessary
- for all ordinal variables: Highest number stands for DK


In [ ]:
columns_analysis["encoding"].value_counts()

In [ ]:
# Create list of names of ordinal columns 

col_names_ordinal = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'ordinal':
    col_names_ordinal.append(columns_analysis.iloc[i, 1])

print(col_names_ordinal)

In [ ]:
for i in range(48):
  print(eppes[col_names_ordinal[i]].value_counts())

In [ ]:
# Drop DKs for ordinal variables

for i in range(48):
  eppes = eppes[eppes[col_names_ordinal[i]] != max(eppes[col_names_ordinal[i]])]

# Drop special DKs

eppes = eppes[eppes.qg11_2 < 5]
eppes = eppes[eppes.qg11_6 < 5]
eppes = eppes[eppes.d1 < 11]
eppes = eppes[eppes.d63 < 6]


In [ ]:
for i in range(48):
  print(eppes[col_names_ordinal[i]].value_counts())

## Change Value type from float to object for categoricals

In [ ]:
eppes.dtypes.value_counts()

In [ ]:
# Create list of names of categorical columns 

col_names_categorical = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'categorical':
    col_names_categorical.append(columns_analysis.iloc[i, 1])

col_names_categorical

In [ ]:
# Change datatype of categorical variables to object

eppes[col_names_categorical] = eppes[col_names_categorical].astype('object')

eppes.dtypes.value_counts()

In [103]:
eppes.describe()

,studyno1,studyno2,edition,survey,caseid,uniqid,serialid,tnscntry,country,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,qa1a_1,qa1a_2,qa1a_5,qa1a_6,qa1a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,qa7,qa8_1,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_11,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa18a_1,qa18a_2,qa18a_3,qa19,qb3_1,qb3_2,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd12_1,qd12_2,qe2_1,qe2_2,qg10,qg11_2,qg11_6,d1,d11,d40a,d40b,d46.8,d62_1,d62_2,d63,d72_1,d72_2,d77
count,12187.0,12187.0,12187.0,12187.0,1.218700e+04,1.218700e+04,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,121

In [ ]:
eppes

In [ ]:
eppes_clean = eppes.drop(eppes.columns[0:12], axis = 1)

eppes_clean.to_csv('eppes_clean.csv')

In [105]:
eppes_clean.describe()

,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,qa1a_1,qa1a_2,qa1a_5,qa1a_6,qa1a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,qa7,qa8_1,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_11,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa18a_1,qa18a_2,qa18a_3,qa19,qb3_1,qb3_2,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd12_1,qd12_2,qe2_1,qe2_2,qg10,qg11_2,qg11_6,d1,d11,d40a,d40b,d46.8,d62_1,d62_2,d63,d72_1,d72_2,d77
count,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000,12187.000000
mean,0.055223,0.037171,0.060228,0.048330,0.037089,0.026750,0.038566,0.042833,0.007713,0.046607,0.025601,0.024863,0.040617,0.046771,0.0